## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy import stats

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

## Load Data

In [ ]:
# Load pre-period features
features_df = pd.read_csv('../data/store_preperiod_features.csv')

# Load full sales data
df = pd.read_csv('../data/store_weekly_sales.csv')

print(f"Loaded features for {len(features_df)} stores")
print(f"Loaded {len(df):,} store-week observations")
print(f"\nFeatures available:")
print(features_df.columns.tolist())
print(f"\nFirst few rows:")
features_df.head()

## Prepare Features for Clustering

We'll use three pre-period features:
- **Slope**: Sales trend (growth/decline)
- **Average Sales**: Store size/scale
- **Volatility**: Sales variability (standard deviation)

In [ ]:
# Select features for clustering
clustering_features = ['pre_slope', 'pre_avg_sales', 'pre_sales_std']

X = features_df[clustering_features].values

print("Features for clustering:")
print(features_df[clustering_features].describe())

# Check for any missing values
print(f"\nMissing values: {features_df[clustering_features].isnull().sum().sum()}")

## Standardize Features

Standardization is critical because:
- Features have different scales (slope in $, sales in $10,000s)
- KMeans is sensitive to feature magnitudes
- Ensures each feature contributes equally to distance calculations

In [ ]:
# Standardize features (mean=0, std=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create DataFrame for easier analysis
X_scaled_df = pd.DataFrame(
    X_scaled, 
    columns=[f'{col}_scaled' for col in clustering_features],
    index=features_df.index
)

print("Standardized features:")
print(X_scaled_df.describe())

# Verify standardization
print(f"\nMeans (should be ~0): {X_scaled.mean(axis=0)}")
print(f"Std devs (should be ~1): {X_scaled.std(axis=0)}")

## Apply KMeans Clustering (k=3)

We choose 3 clusters to represent:
- **Declining stores**: Negative pre-period slope
- **Stable stores**: Near-zero pre-period slope
- **Growing stores**: Positive pre-period slope

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Fit KMeans
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Add cluster assignments to features DataFrame
features_df['cluster'] = clusters

print(f"KMeans clustering complete with {n_clusters} clusters")
print(f"\nCluster sizes:")
print(features_df['cluster'].value_counts().sort_index())

# Show inertia (sum of squared distances to nearest cluster center)
print(f"\nInertia: {kmeans.inertia_:.2f}")

## Analyze Cluster Characteristics

Examine the average features within each cluster to understand what makes them distinct.

In [ ]:
# Calculate cluster statistics
cluster_stats = features_df.groupby('cluster')[clustering_features].agg(['mean', 'std', 'min', 'max'])

print("Cluster Characteristics:")
print("="*80)
print(cluster_stats)

# Also show cluster centers (in original scale)
print("\nCluster Centers (original scale):")
print("="*80)
cluster_centers_original = scaler.inverse_transform(kmeans.cluster_centers_)
centers_df = pd.DataFrame(
    cluster_centers_original,
    columns=clustering_features,
    index=[f'Cluster {i}' for i in range(n_clusters)]
)
print(centers_df)

## Label Clusters Based on Trend Direction

Assign meaningful names based on average pre-period slope.

In [ ]:
# Calculate mean slope for each cluster
cluster_slopes = features_df.groupby('cluster')['pre_slope'].mean().sort_values()

print("Mean slope by cluster:")
print(cluster_slopes)

# Create mapping based on slope ordering
# Cluster with most negative slope = Declining
# Cluster with most positive slope = Growing  
# Middle cluster = Stable
slope_order = cluster_slopes.index.tolist()

cluster_labels = {
    slope_order[0]: 'Declining',
    slope_order[1]: 'Stable',
    slope_order[2]: 'Growing'
}

# Add labels to DataFrame
features_df['segment'] = features_df['cluster'].map(cluster_labels)

print("\nCluster Labels:")
for cluster, label in cluster_labels.items():
    n_stores = (features_df['cluster'] == cluster).sum()
    avg_slope = features_df[features_df['cluster'] == cluster]['pre_slope'].mean()
    print(f"  Cluster {cluster} → '{label}': {n_stores} stores, avg slope = {avg_slope:.2f} $/week")

# Show segment distribution
print("\nSegment distribution:")
print(features_df['segment'].value_counts())

## Segment Summary Table

Create a comprehensive summary showing key characteristics of each segment.

In [ ]:
# Create comprehensive segment summary
segment_summary = features_df.groupby('segment').agg({
    'store_id': 'count',
    'pre_slope': 'mean',
    'pre_avg_sales': 'mean',
    'pre_sales_std': 'mean',
    'pre_cv': 'mean'
}).round(2)

# Rename columns for clarity
segment_summary.columns = [
    'Number of Stores',
    'Avg Weekly Slope ($/week)',
    'Avg Weekly Sales ($)',
    'Avg Sales Volatility ($)',
    'Avg Coefficient of Variation'
]

# Reorder segments
segment_order = ['Declining', 'Stable', 'Growing']
segment_summary = segment_summary.reindex(segment_order)

print("Segment Summary Table")
print("="*100)
print(segment_summary)
print()

# Add percentage of total stores
total_stores = segment_summary['Number of Stores'].sum()
segment_summary['% of Total'] = (segment_summary['Number of Stores'] / total_stores * 100).round(1)

print("\nSegment Summary with Percentages")
print("="*100)
print(segment_summary)

## Business Interpretation of Segments

### 📉 Declining Segment
**Characteristics:**
- **Negative pre-period sales slope**: These stores were experiencing declining sales trends before the pilot
- **Sales trajectory**: Downward trend indicates potential market challenges, increased competition, or operational issues
- **Business implication**: These stores may be struggling and could benefit most from interventions like the new layout

**Why this matters for A/B testing:**
- If treatment is effective, we expect to see trend reversal or slower decline
- Comparing treatment vs control within this segment isolates the pilot effect from the underlying decline
- Without segmentation, declining stores could bias overall results downward

---

### ⚖️ Stable Segment
**Characteristics:**
- **Near-zero pre-period slope**: Sales remained relatively flat before the pilot
- **Sales trajectory**: Steady state indicates mature, established stores with consistent performance
- **Business implication**: These stores represent the "typical" baseline - neither growing nor declining

**Why this matters for A/B testing:**
- Easiest segment to analyze since pre-period trend is minimal
- Treatment effect is most directly observable (any lift is likely due to intervention)
- Provides a clean comparison for traditional pre-post analysis

---

### 📈 Growing Segment
**Characteristics:**
- **Positive pre-period slope**: These stores were already experiencing growth before the pilot
- **Sales trajectory**: Upward trend indicates strong market position, effective operations, or favorable demographics
- **Business implication**: High-performing stores that may have less room for improvement (ceiling effect)

**Why this matters for A/B testing:**
- If treatment is effective, we expect accelerated growth beyond the natural trend
- Comparing treatment vs control within this segment separates pilot impact from organic growth
- Without segmentation, growing stores could inflate overall results and hide true treatment effect

---

### Key Insight
By segmenting stores and analyzing treatment effects **within each segment**, we control for pre-existing trends that could confound our results. This is crucial when stores have heterogeneous growth trajectories.

## Verify Treatment Balance Within Segments

Check that treatment/control are balanced within each segment.

In [ ]:
# Cross-tabulation of segment and treatment
balance_table = pd.crosstab(
    features_df['segment'], 
    features_df['treatment'],
    margins=True
)
balance_table.columns = ['Control', 'Treatment', 'Total']

print("Treatment Balance by Segment:")
print("="*60)
print(balance_table)

# Calculate treatment percentage within each segment
print("\nTreatment % by Segment:")
for segment in ['Declining', 'Stable', 'Growing']:
    segment_data = features_df[features_df['segment'] == segment]
    pct_treatment = segment_data['treatment'].mean() * 100
    print(f"  {segment}: {pct_treatment:.1f}%")

## Visualize Clusters

Plot stores in 2D feature space to visualize cluster separation.

In [ ]:
# Create pairwise scatter plots
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

feature_pairs = [
    ('pre_slope', 'pre_avg_sales'),
    ('pre_slope', 'pre_sales_std'),
    ('pre_avg_sales', 'pre_sales_std')
]

colors = {'Declining': 'red', 'Stable': 'gray', 'Growing': 'green'}

for idx, (feat1, feat2) in enumerate(feature_pairs):
    ax = axes[idx]
    
    for segment, color in colors.items():
        segment_data = features_df[features_df['segment'] == segment]
        ax.scatter(segment_data[feat1], segment_data[feat2], 
                  c=color, label=segment, alpha=0.6, s=50)
    
    ax.set_xlabel(feat1.replace('pre_', '').replace('_', ' ').title(), fontsize=11)
    ax.set_ylabel(feat2.replace('pre_', '').replace('_', ' ').title(), fontsize=11)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.suptitle('Store Segments in Feature Space', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Distribution of Slopes by Segment

In [ ]:
# Box plot of slopes by segment
fig, ax = plt.subplots(figsize=(10, 6))

segments_ordered = ['Declining', 'Stable', 'Growing']
segment_data = [features_df[features_df['segment'] == seg]['pre_slope'] for seg in segments_ordered]

bp = ax.boxplot(segment_data, labels=segments_ordered, patch_artist=True)

# Color the boxes
for patch, color in zip(bp['boxes'], ['red', 'gray', 'green']):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)

ax.set_ylabel('Pre-Period Sales Slope ($ per week)', fontsize=12)
ax.set_xlabel('Segment', fontsize=12)
ax.set_title('Distribution of Pre-Period Slopes by Segment', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
ax.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSlope Statistics by Segment:")
print(features_df.groupby('segment')['pre_slope'].describe())

## Save Segmented Data

## Segment-Level Treatment Effect Analysis

Now that we have stores grouped into segments, we'll estimate treatment effects **within each segment** using a difference-in-differences approach.

In [ ]:
# Define treatment start week
TREATMENT_START_WEEK = 53

# Compute pre and post period average sales for each store
store_period_sales = []

for store_id in features_df['store_id'].unique():
    store_data = df[df['store_id'] == store_id]
    
    # Pre-period average
    pre_sales = store_data[store_data['week'] < TREATMENT_START_WEEK]['sales'].mean()
    
    # Post-period average
    post_sales = store_data[store_data['week'] >= TREATMENT_START_WEEK]['sales'].mean()
    
    # Get segment and treatment info
    segment = features_df[features_df['store_id'] == store_id]['segment'].values[0]
    treatment = features_df[features_df['store_id'] == store_id]['treatment'].values[0]
    
    store_period_sales.append({
        'store_id': store_id,
        'segment': segment,
        'treatment': treatment,
        'pre_sales': pre_sales,
        'post_sales': post_sales,
        'change': post_sales - pre_sales,
        'pct_change': (post_sales - pre_sales) / pre_sales * 100
    })

period_df = pd.DataFrame(store_period_sales)

print("Store-Level Pre-Post Sales Summary:")
print(period_df.head(10))

## Compute Segment-Level Treatment Effects

For each segment, we'll calculate:
1. **Pre-post change for treatment stores**
2. **Pre-post change for control stores** 
3. **Difference-in-Differences (DiD)**: Treatment change minus control change

In [ ]:
# Calculate segment-level treatment effects
segment_results = []

for segment in ['Declining', 'Stable', 'Growing']:
    segment_data = period_df[period_df['segment'] == segment]
    
    # Treatment group
    treatment_data = segment_data[segment_data['treatment'] == 1]
    treatment_pre = treatment_data['pre_sales'].mean()
    treatment_post = treatment_data['post_sales'].mean()
    treatment_change = treatment_post - treatment_pre
    treatment_pct = (treatment_change / treatment_pre) * 100
    
    # Control group
    control_data = segment_data[segment_data['treatment'] == 0]
    control_pre = control_data['pre_sales'].mean()
    control_post = control_data['post_sales'].mean()
    control_change = control_post - control_pre
    control_pct = (control_change / control_pre) * 100
    
    # Difference-in-Differences
    did_absolute = treatment_change - control_change
    did_pct = treatment_pct - control_pct
    
    segment_results.append({
        'segment': segment,
        'n_treatment': len(treatment_data),
        'n_control': len(control_data),
        'treatment_pre': treatment_pre,
        'treatment_post': treatment_post,
        'treatment_change': treatment_change,
        'treatment_pct_change': treatment_pct,
        'control_pre': control_pre,
        'control_post': control_post,
        'control_change': control_change,
        'control_pct_change': control_pct,
        'did_absolute': did_absolute,
        'did_pct': did_pct
    })

results_df = pd.DataFrame(segment_results)

print("Segment-Level Treatment Effect Estimates")
print("="*100)
print(results_df[['segment', 'n_treatment', 'n_control', 'treatment_pct_change', 
                  'control_pct_change', 'did_pct']].to_string(index=False))

print("\\n\\nDetailed Results:")
print(results_df.to_string(index=False))

## Bootstrap Confidence Intervals

Use bootstrap resampling to estimate confidence intervals for the treatment effect in each segment.

In [ ]:
def bootstrap_did(segment_data, n_bootstrap=1000, random_state=42):
    """
    Bootstrap the Difference-in-Differences estimate for a segment.
    
    Returns: (point_estimate, lower_ci, upper_ci)
    """
    np.random.seed(random_state)
    
    treatment_stores = segment_data[segment_data['treatment'] == 1]
    control_stores = segment_data[segment_data['treatment'] == 0]
    
    bootstrap_estimates = []
    
    for _ in range(n_bootstrap):
        # Resample treatment stores with replacement
        treatment_sample = treatment_stores.sample(n=len(treatment_stores), replace=True)
        treatment_change = treatment_sample['change'].mean()
        
        # Resample control stores with replacement
        control_sample = control_stores.sample(n=len(control_stores), replace=True)
        control_change = control_sample['change'].mean()
        
        # DiD estimate for this bootstrap sample
        did = treatment_change - control_change
        
        # Convert to percentage
        treatment_pre_mean = treatment_sample['pre_sales'].mean()
        did_pct = (did / treatment_pre_mean) * 100
        
        bootstrap_estimates.append(did_pct)
    
    bootstrap_estimates = np.array(bootstrap_estimates)
    
    # Calculate percentile-based confidence intervals
    point_estimate = np.mean(bootstrap_estimates)
    lower_ci = np.percentile(bootstrap_estimates, 2.5)
    upper_ci = np.percentile(bootstrap_estimates, 97.5)
    
    return point_estimate, lower_ci, upper_ci

# Apply bootstrap to each segment
bootstrap_results = []

print("Bootstrap Confidence Intervals (1000 iterations)")
print("="*70)

for segment in ['Declining', 'Stable', 'Growing']:
    segment_data = period_df[period_df['segment'] == segment]
    
    point_est, lower_ci, upper_ci = bootstrap_did(segment_data, n_bootstrap=1000)
    
    bootstrap_results.append({
        'segment': segment,
        'did_pct': point_est,
        'ci_lower': lower_ci,
        'ci_upper': upper_ci,
        'ci_width': upper_ci - lower_ci
    })
    
    print(f"{segment:12s}: {point_est:6.2f}% [95% CI: {lower_ci:6.2f}%, {upper_ci:6.2f}%]")

bootstrap_df = pd.DataFrame(bootstrap_results)
print("\\n✓ Bootstrap complete!")

## Visualize Segment-Level Results

Create visualizations showing treatment effects by segment with confidence intervals.

In [ ]:
# Treatment effect by segment with confidence intervals
fig, ax = plt.subplots(figsize=(10, 6))

segments = bootstrap_df['segment'].values
did_pcts = bootstrap_df['did_pct'].values
ci_lower = bootstrap_df['ci_lower'].values
ci_upper = bootstrap_df['ci_upper'].values
errors = np.array([did_pcts - ci_lower, ci_upper - did_pcts])

colors_map = {'Declining': 'red', 'Stable': 'gray', 'Growing': 'green'}
bar_colors = [colors_map[seg] for seg in segments]

x_pos = np.arange(len(segments))

# Create bar chart with error bars
bars = ax.bar(x_pos, did_pcts, color=bar_colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.errorbar(x_pos, did_pcts, yerr=errors, fmt='none', color='black', 
            capsize=8, capthick=2, linewidth=2, label='95% CI')

# Add horizontal line at 0
ax.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)

# Add value labels on bars
for i, (segment, did_pct, lower, upper) in enumerate(zip(segments, did_pcts, ci_lower, ci_upper)):
    ax.text(i, did_pct + (ci_upper[i] - ci_lower[i])/2 + 1, 
            f'{did_pct:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Formatting
ax.set_xlabel('Segment', fontsize=13, fontweight='bold')
ax.set_ylabel('Treatment Effect (% Lift)', fontsize=13, fontweight='bold')
ax.set_title('Segment-Level Treatment Effects with 95% Confidence Intervals', 
             fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(segments, fontsize=12)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\\nInterpretation:")
print("- Error bars show 95% confidence intervals from bootstrap resampling")
print("- If error bars don't cross zero, the effect is statistically significant at p<0.05")

In [ ]:
# Pre-Post comparison by segment and treatment group
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for idx, segment in enumerate(['Declining', 'Stable', 'Growing']):
    ax = axes[idx]
    
    # Get data for this segment
    segment_data = period_df[period_df['segment'] == segment]
    
    # Treatment group
    treatment_data = segment_data[segment_data['treatment'] == 1]
    treatment_pre = treatment_data['pre_sales'].mean()
    treatment_post = treatment_data['post_sales'].mean()
    
    # Control group
    control_data = segment_data[segment_data['treatment'] == 0]
    control_pre = control_data['pre_sales'].mean()
    control_post = control_data['post_sales'].mean()
    
    # Set up x positions
    x = np.array([0, 1])
    width = 0.35
    
    # Plot bars
    ax.bar(x - width/2, [treatment_pre, treatment_post], width, 
           label='Treatment', color='orange', alpha=0.7, edgecolor='black')
    ax.bar(x + width/2, [control_pre, control_post], width, 
           label='Control', color='blue', alpha=0.7, edgecolor='black')
    
    # Add connecting lines to show change
    ax.plot([0-width/2, 1-width/2], [treatment_pre, treatment_post], 
            'o-', color='orange', linewidth=2, markersize=8, alpha=0.5)
    ax.plot([0+width/2, 1+width/2], [control_pre, control_post], 
            'o-', color='blue', linewidth=2, markersize=8, alpha=0.5)
    
    # Formatting
    ax.set_ylabel('Average Weekly Sales ($)', fontsize=11)
    ax.set_title(f'{segment} Segment', fontsize=12, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(['Pre-Period', 'Post-Period'], fontsize=10)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

plt.suptitle('Pre-Post Sales Comparison by Segment', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\\nNote: Steeper slopes for treatment (orange) vs control (blue) indicate positive treatment effects")

In [ ]:
# Save features with cluster assignments
output_path = '../data/store_segments.csv'
features_df.to_csv(output_path, index=False)

print(f"Saved segmented data to: {output_path}")
print(f"\nColumns saved:")
for col in features_df.columns:
    print(f"  - {col}")

print("\n✓ Segmentation complete!")
print(f"\nNext steps:")
print("  1. Compute segment-level treatment effects")
print("  2. Aggregate for overall estimate")
print("  3. Compare to naive pre-post analysis")